In [ ]:
# block1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
from shapely.geometry import box
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score
import os
import requests
import zipfile
import shutil
import glob
import math
from google.colab import drive

drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')
temp_dir = os.path.join(base_dir, 'Temp_Processing')

for d in [data_dir, map_dir, temp_dir]:
    if not os.path.exists(d): os.makedirs(d)

train_file = os.path.join(data_dir, 'landslides.csv')

shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Muncipalities_polygon.shp'

# CRU TS Links
cru_urls = {
    "tmin_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmin_2010-2019.zip",
    "tmin_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmin_2020-2024.zip",
    "tmax_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmax_2010-2019.zip",
    "tmax_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_tmax_2020-2024.zip",
    "prec_2010": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_prec_2010-2019.zip",
    "prec_2020": "https://geodata.ucdavis.edu/climate/worldclim/2_1/hist/cts4.09/wc2.1_cruts4.09_2.5m_prec_2020-2024.zip"
}

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

# Grid
GRID_RES = 0.00416
BBOX = (10.0, 46.0, 13.0, 47.5)
geo_bbox = gpd.GeoDataFrame({'geometry': box(*BBOX)}, index=[0], crs="EPSG:4326")

# MODELL TRAINING
print("\n 1. MODELL TRAINING")
if not os.path.exists(train_file):
    print(f"FEHLER: Lade '{os.path.basename(train_file)}' nach '{data_dir}'!")
    exit()

df_train = pd.read_csv(train_file)
# Datum suchen für Validierung
date_col = None
for col in df_train.columns:
    if any(x in col.lower() for x in ['date', 'jahr', 'year']):
        date_col = col; break
print(f"Datumsspalte: {date_col}")

# Features vorbereiten
for f in features:
    if f not in df_train.columns: df_train[f] = 0

rf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_leaf=2,
                            class_weight='balanced', max_features='sqrt', n_jobs=-1, random_state=42)
rf.fit(df_train[features].fillna(0), df_train['Target'])
print("Modell trainiert.")

# GITTER & TOPO
print("\n2. GITTER & TOPOGRAPHIE")
grid_csv = os.path.join(temp_dir, 'Grid_Topo_Only.csv')

if not os.path.exists(grid_csv):
    # Gitter erstellen
    lon = np.arange(BBOX[0], BBOX[2], GRID_RES)
    lat = np.arange(BBOX[1], BBOX[3], GRID_RES)
    xx, yy = np.meshgrid(lon, lat)
    df_grid = pd.DataFrame({'Longitude': xx.flatten(), 'Latitude': yy.flatten()})
    print(f"Gitter: {len(df_grid)} Punkte")

    # Elevation Download
    elev_zip = os.path.join(temp_dir, "elev.zip")
    elev_tif = os.path.join(temp_dir, "wc2.1_30s_elev.tif")

    if not os.path.exists(elev_tif):
        print("Lade DEM...")
        with requests.get("https://geodata.ucdavis.edu/climate/worldclim/2_1/base/wc2.1_30s_elev.zip", stream=True) as r:
            with open(elev_zip, 'wb') as f: shutil.copyfileobj(r.raw, f)
        with zipfile.ZipFile(elev_zip, 'r') as z:
            z.extract([n for n in z.namelist() if n.endswith('.tif')][0], temp_dir)
            os.rename(os.path.join(temp_dir, [n for n in z.namelist() if n.endswith('.tif')][0]), elev_tif)

    # Slope/Aspect berechnen
    print("Berechne Slope/Aspect...")
    with rasterio.open(elev_tif) as src:
        coords = [(r.Longitude, r.Latitude) for _, r in df_grid.iterrows()]
        df_grid['Elevation'] = [x[0] for x in src.sample(coords)]

        # Mask für Gradienten
        out_img, out_transform = mask(src, geo_bbox.geometry, crop=True)
        data = out_img[0].astype('float32')
        res_x, res_y = out_transform[0], -out_transform[4]
        scale_x = 111320 * math.cos(46.5*math.pi/180) * res_x
        scale_y = 111320 * res_y

        dy, dx = np.gradient(data)
        slope = np.degrees(np.arctan(np.sqrt((dx/scale_x)**2 + (dy/scale_y)**2)))
        aspect = (np.degrees(np.arctan2(dy, -dx)) + 360) % 360

        # In Temp-Tifs schreiben für Sampling
        meta = src.meta.copy()
        meta.update({"height": data.shape[0], "width": data.shape[1], "transform": out_transform})
        with rasterio.open(os.path.join(temp_dir, 'slope.tif'), 'w', **meta) as dst: dst.write(slope, 1)
        with rasterio.open(os.path.join(temp_dir, 'aspect.tif'), 'w', **meta) as dst: dst.write(aspect, 1)

    with rasterio.open(os.path.join(temp_dir, 'slope.tif')) as s:
        df_grid['Slope'] = [x[0] for x in s.sample(coords)]
    with rasterio.open(os.path.join(temp_dir, 'aspect.tif')) as a:
        df_grid['Aspect'] = [x[0] for x in a.sample(coords)]

    df_grid.to_csv(grid_csv, index=False)
    print("Topo fertig.")
else:
    print("Lade existierendes Topo-Grid...")
    df_grid = pd.read_csv(grid_csv)


# DATEN (2015-2024)
print("\n3. DATEN VERARBEITUNG (2015-2024)")
def process_cru_downloads():
    all_data = {'tmin': {}, 'tmax': {}, 'prec': {}}

    for key, url in cru_urls.items():
        var_type = key.split('_')[0]
        zip_path = os.path.join(temp_dir, f"{key}.zip")
        extract_path = os.path.join(temp_dir, key)

        # Download
        if not os.path.exists(extract_path):
            print(f"Lade {key}...")
            if not os.path.exists(zip_path):
                with requests.get(url, stream=True) as r:
                    with open(zip_path, 'wb') as f: shutil.copyfileobj(r.raw, f)
            with zipfile.ZipFile(zip_path, 'r') as z: z.extractall(extract_path)
            os.remove(zip_path)

        # Lesen & Croppen
        tifs = glob.glob(f"{extract_path}/*.tif")
        print(f"  Verarbeite {len(tifs)} Dateien für {key}...")

        for tif in tifs:
            try:
                # Dateiname parsen (manchmal '..._2010-01.tif')
                date_part = os.path.basename(tif).replace('.tif','').split('_')[-1]
                y, m = map(int, date_part.split('-'))

                if 2015 <= y <= 2024:
                    with rasterio.open(tif) as src:
                        out_img, out_trans = mask(src, geo_bbox.geometry, crop=True)
                        all_data[var_type][f"{m:02d}_{y}"] = (out_img[0], out_trans, src.crs, src.meta)
            except: continue

        shutil.rmtree(extract_path) # Aufräumen
    return all_data

# Daten laden
cru_data = process_cru_downloads()
print("Rohdaten im Speicher.")

# Klima berechnen
print("Berechne Durchschnitte & BIOs...")
bio_values = {}
# Referenz-Meta für das Upscaling später
ref_meta = None

# Mittelwerte berechnen
monthly_means = {'tmin': [], 'tmax': [], 'prec': []}

for month in range(1, 13):
    m_str = f"{month:02d}"
    for var in ['tmin', 'tmax', 'prec']:
        arrays = []
        for y in range(2015, 2025):
            k = f"{m_str}_{y}"
            if k in cru_data[var]:
                data, trans, crs, meta = cru_data[var][k]
                arrays.append(data)
                if ref_meta is None: ref_meta = meta.copy(); ref_meta.update({"height": data.shape[0], "width": data.shape[1], "transform": trans})

        if arrays:
            monthly_means[var].append(np.mean(np.stack(arrays), axis=0))
        else:
            print(f"WARNUNG: Keine Daten für {var} Monat {m}")

# Stapeln (12 Monate)
tmin = np.stack(monthly_means['tmin'])
tmax = np.stack(monthly_means['tmax'])
prec = np.stack(monthly_means['prec'])

# BIOs
bio_values['BIO01_Historical_Mean'] = np.mean((tmax + tmin) / 2, axis=0)
bio_values['BIO05_Historical_Max'] = np.max(tmax, axis=0)
bio_values['BIO06_Historical_Min'] = np.min(tmin, axis=0)
bio_values['BIO12_Historical_Prec'] = np.sum(prec, axis=0)
bio_values['BIO13_Historical_Prec'] = np.max(prec, axis=0)
prec_mean = np.mean(prec, axis=0)
with np.errstate(divide='ignore', invalid='ignore'):
    bio15 = (np.std(prec, axis=0) / prec_mean) * 100
    bio15[prec_mean == 0] = 0
bio_values['BIO15_Historical_Prec'] = bio15

print("BIOs berechnet. Starte Upscaling & Mapping auf Grid...")
# Upscaling und Zuweisung an Grid
coords = [(r.Longitude, r.Latitude) for _, r in df_grid.iterrows()]

for name, grid_data in bio_values.items():
    # MemoryFile für das Upscaling
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**ref_meta) as dataset:
            dataset.write(grid_data.astype('float32'), 1)

            # Upscaling (Faktor 10 für ~500m)
            upscale = 10
            new_h = int(dataset.height * upscale)
            new_w = int(dataset.width * upscale)

            data_up = dataset.read(
                1, out_shape=(1, new_h, new_w), resampling=Resampling.bilinear
            )

            # Neue Transformation berechnen
            trans_up = dataset.transform * dataset.transform.scale(
                (dataset.width / data_up.shape[-1]),
                (dataset.height / data_up.shape[-2])
            )

            # Temporär nue
            up_meta = dataset.meta.copy()
            up_meta.update({"height": new_h, "width": new_w, "transform": trans_up})

            with rasterio.io.MemoryFile() as up_mem:
                with up_mem.open(**up_meta) as up_ds:
                    up_ds.write(data_up, 1)
                    # Samplen für die Gitterpunkte
                    df_grid[name] = [x[0] for x in up_ds.sample(coords)]
    print(f"  -> {name} fertig.")

# Speichern
df_grid.to_csv(os.path.join(map_dir, 'Grid_Full_Data_2015_2024.csv'), index=False)


# VORHERSAGE & VALIDIERUNG
print("\n 4. MAPPING & VALIDIERUNG")

# Vorhersage
print("Berechne Risiko...")
probs = rf.predict_proba(df_grid[features].fillna(0))[:, 1]
df_grid['Landslide_Probability'] = probs

# Speichern
df_grid.to_csv(os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv'), index=False)

# Basis-Plot
plt.figure(figsize=(12, 10))
plt.scatter(df_grid.Longitude, df_grid.Latitude, c=df_grid.Landslide_Probability, cmap='Reds', s=2, marker='s', vmin=0, vmax=1)
plt.colorbar(label='Risk Probability')
plt.title('Historical Landslide Susceptibility (2015-2024)')
plt.savefig(os.path.join(map_dir, 'Map_2015_2024.png'), dpi=300)
plt.close()

# Validierung (Jahres-Overlay)
if date_col:
    df_train['Year'] = pd.to_datetime(df_train[date_col], errors='coerce').dt.year
    years = sorted(df_train[df_train.Target == 1]['Year'].dropna().unique().astype(int))

    print(f"Erstelle Validierungskarten für: {years}")
    for year in years:
        if year < 2015: continue

        events = df_train[(df_train.Target == 1) & (df_train.Year == year)]
        if len(events) == 0: continue

        plt.figure(figsize=(12, 10))
        plt.scatter(df_grid.Longitude, df_grid.Latitude, c=df_grid.Landslide_Probability, cmap='Reds', s=2, marker='s', vmin=0, vmax=1, alpha=0.6)
        plt.colorbar(label='Risk Probability')

        # Events
        plt.scatter(events.Longitude, events.Latitude, c='black', marker='x', s=100, linewidth=2, label=f'Events {year}')

        plt.title(f"Validation Year {year} (Events vs 2015-2024 Risk)")
        plt.legend()
        plt.savefig(os.path.join(map_dir, f'Validation_Map_{year}.png'), dpi=300)
        plt.close()
        print(f"  -> Jahr {year} fertig.")

# GEMEINDE ANALYSE (Auf Basis der 2015-2024 Karte)
print("\n 5. GEMEINDE ANALYSE")
if os.path.exists(shapefile_path):
    try:
        print("Lade Shapefile...")
        gdf_gem = gpd.read_file(shapefile_path)

        # CRS Check
        if gdf_gem.crs != "EPSG:4326":
            print(f"Transformiere Shapefile von {gdf_gem.crs} nach EPSG:4326...")
            gdf_gem = gdf_gem.to_crs("EPSG:4326")

        # Grid in GeoDataFrame umwandeln
        print("Wandle Grid in Geometrie um...")
        gdf_points = gpd.GeoDataFrame(
            df_grid,
            geometry=gpd.points_from_xy(df_grid.Longitude, df_grid.Latitude),
            crs="EPSG:4326"
        )

        print("Spatial Join (Welcher Punkt liegt in welcher Gemeinde?)...")
        joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

        # Namens-Spalte
        name_col = 'NAME_DE'

        print(f"Gruppiere nach: {name_col}")

        # Aggregieren: Durchschnittliches Risiko pro Gemeinde
        risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
        risk_gem.columns = [name_col, 'Mean_Hazard']

        # Zurück ins Shapefile mergen für die Karte
        gdf_final = gdf_gem.merge(risk_gem, on=name_col)

        # Plotten
        plt.figure(figsize=(12, 10))
        ax = plt.gca()
        gdf_final.plot(column='Mean_Hazard', ax=ax, cmap='OrRd', legend=True,
                       legend_kwds={'label': "Durchschnittl. Gefährdung (2015-2024 Climate)"})
        plt.title("Gemeinde-Gefährdungskarte")
        plt.savefig(os.path.join(map_dir, "Municipality_Risk_Map.png"), dpi=300)
        plt.close()

        # CSV Export
        out_csv = os.path.join(map_dir, "Municipality_Risk_Table.csv")
        gdf_final[[name_col, 'Mean_Hazard']].to_csv(out_csv, index=False)
        print(f"Gemeinde-Daten gespeichert: {out_csv}")

    except Exception as e:
        print(f"Fehler bei Gemeinde-Analyse: {e}")
else:
    print(f"Shapefile nicht gefunden unter: {shapefile_path}")

print(f"\nFERTIG! Alles gespeichert in: {map_dir}")

In [ ]:
#block 3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from sklearn.ensemble import RandomForestClassifier
import os
from google.colab import drive

drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')
temp_dir = os.path.join(base_dir, 'Temp_Processing')

shapefile_path = '/content/drive/MyDrive/Landslides/Data/FME_11060556_1767623643023_2240/Municipalities_polygon.shp'

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

ssp_urls = {
    'SSP126': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp126/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp126_2081-2100.tif",
    'SSP370': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp370/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp370_2081-2100.tif",
    'SSP585': "https://geodata.ucdavis.edu/cmip6/30s/MPI-ESM1-2-HR/ssp585/wc2.1_30s_bioc_MPI-ESM1-2-HR_ssp585_2081-2100.tif"
}

bio_map = {1:'BIO01_Historical_Mean', 5:'BIO05_Historical_Max', 6:'BIO06_Historical_Min',
           12:'BIO12_Historical_Prec', 13:'BIO13_Historical_Prec', 15:'BIO15_Historical_Prec'}

# 1. MODELL, GRID & SHAPEFILE LADEN
# Shapefile laden (für den Hintergrund)
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path)
    if gdf_gem.crs != "EPSG:4326": gdf_gem = gdf_gem.to_crs("EPSG:4326")
else:
    print("ACHTUNG: Shapefile nicht gefunden! Karten werden ohne Grenzen erstellt.")
    gdf_gem = None

# Grid laden
grid_topo_path = os.path.join(temp_dir, 'Grid_Topo_Only.csv')
if not os.path.exists(grid_topo_path):
    print("FEHLER: 'Grid_Topo_Only.csv' fehlt. Bitte Teil 1 (CRU Skript) laufen lassen.")
    exit()

df_grid_base = pd.read_csv(grid_topo_path)

# Modell trainieren
train_file = os.path.join(data_dir, 'landslides.csv')
df_train = pd.read_csv(train_file)
for f in features:
    if f not in df_train.columns: df_train[f] = 0

print("Trainiere Modell...")
rf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_leaf=2,
                            class_weight='balanced', max_features='sqrt', n_jobs=-1, random_state=42)
rf.fit(df_train[features].fillna(0), df_train['Target'])


# 2. ZUKUNFTS-SZENARIEN (Prediction & Mapping)
results = {}

for name, url in ssp_urls.items():
    print(f"\n--- Verarbeite {name} ---")
    df_scen = df_grid_base.copy()
    coords = [(r.Longitude, r.Latitude) for _, r in df_scen.iterrows()]

    print("Streame Klimadaten...")
    with rasterio.open(url) as src:
        for b_idx, col in bio_map.items():
            if b_idx <= src.count:
                df_scen[col] = [x[0] for x in src.sample(coords, indexes=b_idx)]
            else:
                df_scen[col] = 0

    print("Vorhersage...")
    probs = rf.predict_proba(df_scen[features].fillna(0))[:, 1]
    df_scen['Landslide_Probability'] = probs

    # Speichern CSV
    df_scen.to_csv(os.path.join(map_dir, f'Result_Grid_{name}.csv'), index=False)

    # PLOTTING MIT HINTERGRUND
    print(f"Erstelle Karte für {name}...")
    fig, ax = plt.subplots(figsize=(12, 10))

    # 1. Die Risikokarte (Scatter)
    sc = ax.scatter(df_scen.Longitude, df_scen.Latitude,
                    c=df_scen.Landslide_Probability,
                    cmap='Reds', vmin=0, vmax=1,
                    s=12, marker='s', zorder=1)

    # 2. Gemeindegrenzen darüberlegen (Overlay)
    if gdf_gem is not None:
        gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.3, alpha=0.6, zorder=2)

    plt.colorbar(sc, label='Probability')
    plt.title(f'Future Projection: {name} (2081-2100)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    out_png = os.path.join(map_dir, f'Map_{name}.png')
    plt.savefig(out_png, dpi=300)
    plt.close()

    # Für SSP3 das DF für die Gemeinde-Analyse speichern
    if name == 'SSP370':
        results['SSP3'] = df_scen


# 3. GEMEINDE ANALYSE
print("\n GEMEINDE ANALYSE (SSP3)")
if 'SSP3' in results and gdf_gem is not None:
    try:
        df_ssp3 = results['SSP3']
        gdf_points = gpd.GeoDataFrame(df_ssp3, geometry=gpd.points_from_xy(df_ssp3.Longitude, df_ssp3.Latitude), crs="EPSG:4326")

        print("Spatial Join...")
        joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

        # Name finden
        name_col = 'NAME_DE'
        if name_col not in joined.columns:
            cands = [c for c in joined.columns if 'NAME' in c]
            name_col = cands[0] if cands else joined.columns[0]

        print(f"Aggregiere nach {name_col}...")
        risk_gem = joined.groupby(name_col)['Landslide_Probability'].mean().reset_index()
        risk_gem.columns = [name_col, 'Mean_Hazard_SSP3']

        gdf_final = gdf_gem.merge(risk_gem, on=name_col)

        # Plot (die Gemeinden eingefärbt)
        fig, ax = plt.subplots(figsize=(12, 10))
        gdf_final.plot(column='Mean_Hazard_SSP3', ax=ax, cmap='OrRd',
                       legend=True, legend_kwds={'label': "Durchschnittl. Gefährdung (SSP3)"},
                       edgecolor='black', linewidth=0.2)

        plt.title("Gefährdung pro Gemeinde (Szenario SSP3-7.0)")
        plt.savefig(os.path.join(map_dir, "Municipality_Risk_SSP3.png"), dpi=300)

        gdf_final[[name_col, 'Mean_Hazard_SSP3']].to_csv(os.path.join(map_dir, "Municipality_SSP3_Table.csv"), index=False)
        print("Gemeinde SSP3 Karte fertig.")

    except Exception as e:
        print(f"Fehler Gemeinde Analyse: {e}")

print("\nFERTIG!")

In [ ]:
# block13
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import os

base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
train_file = os.path.join(data_dir, 'landslides.csv')

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

df = pd.read_csv(train_file)
X = df[features]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Searching for best model parameters (5-Fold CV)...")

param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_leaf': [5, 8, 10, 15],
    'n_estimators': [200, 300],
    'max_features': ['sqrt']
}

rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    return_train_score=True,
    n_jobs=-1
)

grid.fit(X_train, y_train)

results = pd.DataFrame(grid.cv_results_)
results['overfitting_gap'] = results['mean_train_score'] - results['mean_test_score']

valid_models = results[results['overfitting_gap'] < 0.16]

if not valid_models.empty:
    best_row = valid_models.sort_values(by='mean_test_score', ascending=False).iloc[0]
    best_params = best_row['params']
    print("\nSUCCESS: Found a model satisfying gap < 16%.")
else:
    print("\nWARNING: No model satisfied gap < 16%. Selecting model with lowest gap.")
    best_row = results.sort_values(by='overfitting_gap', ascending=True).iloc[0]
    best_params = best_row['params']

print("-" * 60)
print("BEST HYPERPARAMETERS FOUND")
print("-" * 60)
print(f"Max Depth:        {best_params['max_depth']}")
print(f"Min Samples Leaf: {best_params['min_samples_leaf']}")
print(f"N Estimators:     {best_params['n_estimators']}")
print(f"CV Train F1:      {best_row['mean_train_score']:.4f}")
print(f"CV Val F1:        {best_row['mean_test_score']:.4f}")
print(f"Overfitting Gap:  {best_row['overfitting_gap']:.4f}")

final_model = RandomForestClassifier(
    **best_params,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

final_model.fit(X_train, y_train)
y_test_pred = final_model.predict(X_test)
test_f1 = f1_score(y_test, y_test_pred)

print("-" * 60)
print("FINAL TEST SET EVALUATION")
print(f"Test F1-Score: {test_f1:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
# block14
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
from google.colab import drive

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
train_file = os.path.join(data_dir, 'landslides.csv')

# Feature Sets
features_full = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

features_no_aspect = [f for f in features_full if f != 'Aspect']

df = pd.read_csv(train_file)
y = df['Target']

# 1. Evaluate Model WITH Aspect
print("Training Model A: WITH Aspect...")
X_full = df[features_full]
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X_full, y, test_size=0.2, random_state=42, stratify=y)

rf_A = RandomForestClassifier(
    n_estimators=300, max_depth=7, min_samples_leaf=5,
    class_weight='balanced', max_features='sqrt',
    n_jobs=-1, random_state=42
)
rf_A.fit(X_train_A, y_train_A)
y_pred_A = rf_A.predict(X_test_A)

# 2. Evaluate Model WITHOUT Aspect
print("Training Model B: WITHOUT Aspect...")
X_no_aspect = df[features_no_aspect]
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(X_no_aspect, y, test_size=0.2, random_state=42, stratify=y)

rf_B = RandomForestClassifier(
    n_estimators=300, max_depth=7, min_samples_leaf=5,
    class_weight='balanced', max_features='sqrt',
    n_jobs=-1, random_state=42
)
rf_B.fit(X_train_B, y_train_B)
y_pred_B = rf_B.predict(X_test_B)

# 3. Comparison Report
print("\n" + "="*60)
print(f"{'METRIC':<15} {'WITH ASPECT':<15} {'WITHOUT ASPECT':<15} {'DIFFERENCE':<15}")
print("="*60)

metrics = {
    'Accuracy': accuracy_score,
    'Precision': precision_score,
    'Recall': recall_score,
    'F1-Score': f1_score
}

for name, func in metrics.items():
    score_A = func(y_test_A, y_pred_A)
    score_B = func(y_test_B, y_pred_B)
    diff = score_A - score_B
    print(f"{name:<15} {score_A:.4f}          {score_B:.4f}          {diff:+.4f}")

print("-" * 60)
print("Feature Importance (Mit Aspect):")
importances = rf_A.feature_importances_
for i, name in enumerate(features_full):
    if name == 'Aspect':
        print(f"Aspect Importance: {importances[i]:.4f}")

In [ ]:
# block15
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import os
from google.colab import drive

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
data_dir = os.path.join(base_dir, 'Data')
map_dir = os.path.join(base_dir, 'Output_Maps')
train_file = os.path.join(data_dir, 'landslides.csv')
shapefile_path = os.path.join(data_dir, 'FME_11060556_1767623643023_2240/Municipalities_polygon.shp')

# os.makedirs(map_dir, exist_ok=True)

# MODEL TRAINING

features = [
    'Elevation', 'Slope', 'Aspect',
    'BIO01_Historical_Mean', 'BIO05_Historical_Max', 'BIO06_Historical_Min',
    'BIO12_Historical_Prec', 'BIO13_Historical_Prec', 'BIO15_Historical_Prec'
]

df = pd.read_csv(train_file)
X = df[features]
y = df['Target']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training Optimized Model (Depth=7, MinSamples=5)...")
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=5,
    class_weight='balanced',
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)

# METRICS REPORT
y_test_pred = rf.predict(X_test)
y_train_pred = rf.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred)
rec = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print("\n" + "="*30)
print("MODEL REPORT")
print("1. OVERFITTING CHECK")
print(f"Train Acc: {train_acc:.4f}")
print(f"Test Acc:  {test_acc:.4f}")
print(f"Gap:       {train_acc - test_acc:.4f}")
print("2. TEST METRICS")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-Score:  {f1:.4f}")
print("3. CLASSIFICATION REPORT")
print(classification_report(y_test, y_test_pred))

# Save Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Test Set)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig(os.path.join(map_dir, 'confusion_matrix.png'), dpi=300)
plt.close()

# Save Feature Importance
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [features[i] for i in indices]

plt.figure(figsize=(10, 6))
plt.barh(range(len(features)), importances[indices], align='center', color='#4c72b0')
plt.yticks(range(len(features)), names)
plt.title('Feature Importance (RF Optimized)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(os.path.join(map_dir, 'feature_importance.png'), dpi=300)
plt.close()

# Text Output for Features
print("4. FEATURE IMPORTANCE")
for i in range(len(features)):
    print(f"{i+1}. {names[i]:<25} {importances[indices[i]]:.4f}")

# UPDATE GRID FILES & GENERATE STATS
files_to_update = {
    'Historical': 'Risk_Map_Data_2015_2024.csv',
    'SSP1-2.6': 'Result_Grid_SSP126.csv',
    'SSP3-7.0': 'Result_Grid_SSP370.csv',
    'SSP5-8.5': 'Result_Grid_SSP585.csv'
}

data_frames = {}
high_risk_shares = {}

for name, filename in files_to_update.items():
    path = os.path.join(map_dir, filename)
    if os.path.exists(path):
        print(f"Updating {name} ({filename})...")
        df_grid = pd.read_csv(path)

        # Check if features exist
        if all(f in df_grid.columns for f in features):
            # PREDICT NEW PROBABILITIES
            probs = rf.predict_proba(df_grid[features])[:, 1]
            df_grid['Landslide_Probability'] = probs

            # Save back to drive
            df_grid.to_csv(path, index=False)
            data_frames[name] = df_grid

            # Store stats for plotting
            high_risk_shares[name] = (probs > 0.5).mean()
        else:
            print(f"  -> ERROR: Missing features in {filename}. Skipping.")
    else:
        print(f"  -> WARNING: File {filename} not found. Cannot update or plot.")

# PLOT: Risk Density
if data_frames:
    plt.figure(figsize=(10, 6))
    for name, df_scen in data_frames.items():
        sns.kdeplot(df_scen['Landslide_Probability'], label=name, fill=True, alpha=0.1)

    plt.title('Risk Distribution Density by Scenario')
    plt.xlabel('Landslide Probability')
    plt.xlim(0, 1)
    plt.legend()
    plt.savefig(os.path.join(map_dir, 'risk_density_distribution.png'), dpi=300)
    plt.close()

# PLOT: High Risk Share
if high_risk_shares:
    plt.figure(figsize=(8, 5))
    names_bar = list(high_risk_shares.keys())
    values = [high_risk_shares[n] * 100 for n in names_bar] # Convert to %
    plt.bar(names_bar, values, color=['gray', 'green', 'orange', 'red'])
    plt.ylabel('Share of High-Risk Area (%)')
    plt.title('Share of Area with Probability > 0.5')
    for i, v in enumerate(values):
        plt.text(i, v + 0.1, f"{v:.1f}%", ha='center')
    plt.savefig(os.path.join(map_dir, 'high_risk_share_comparison.png'), dpi=300)
    plt.close()

# MAP GENERATION (MANUAL LAYOUT)
print("GENERATING MAPS")
# MANUAL CONFIG
LEFT_MARGIN = 0.8
RIGHT_MARGIN = 1.2
BAR_GAP = 0.4
BAR_WIDTH = 0.25

colors = ["#009900", "#FFFF00", "#FF0000"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("TrafficLight", colors)

# Load Shapefile & Calc Geometry
if os.path.exists(shapefile_path):
    gdf_gem = gpd.read_file(shapefile_path).to_crs("EPSG:4326")
    minx, miny, maxx, maxy = gdf_gem.total_bounds

    zoom_xlim = (minx - 0.02, maxx + 0.02)
    zoom_ylim = (miny - 0.02, maxy + 0.02)

    zoom_width = zoom_xlim[1] - zoom_xlim[0]
    zoom_height = zoom_ylim[1] - zoom_ylim[0]
    mean_lat = (zoom_ylim[0] + zoom_ylim[1]) / 2
    geo_factor = 1 / np.cos(np.deg2rad(mean_lat))

    target_aspect = (zoom_width / zoom_height) / geo_factor

    map_height_in = 6
    map_width_in = map_height_in * target_aspect

    total_height = map_height_in + 1.0
    total_width = LEFT_MARGIN + map_width_in + BAR_GAP + BAR_WIDTH + RIGHT_MARGIN
    figsize_manual = (total_width, total_height)
else:
    print("Shapefile not found.")
    exit()

def plot_final_layout(data, col_name, title, filename, is_shapefile=False, v_max=1.0, legend_label='', overlay_events=False):
    fig = plt.figure(figsize=figsize_manual)

    # Axes Positions
    ax_left = LEFT_MARGIN / total_width
    ax_bottom = 0.5 / total_height
    ax_width = map_width_in / total_width
    ax_height = map_height_in / total_height

    cbar_left = (LEFT_MARGIN + map_width_in + BAR_GAP) / total_width
    cbar_width = BAR_WIDTH / total_width

    ax = fig.add_axes([ax_left, ax_bottom, ax_width, ax_height])
    cax = fig.add_axes([cbar_left, ax_bottom, cbar_width, ax_height])

    # Plot Content
    if is_shapefile:
        im = data.plot(column=col_name, ax=ax, cmap=custom_cmap, vmin=0, vmax=v_max,
                       edgecolor='black', linewidth=0.3).collections[0]
    else:
        # Pixel Map
        mask = (data.Longitude >= zoom_xlim[0]) & (data.Longitude <= zoom_xlim[1]) & \
               (data.Latitude >= zoom_ylim[0]) & (data.Latitude <= zoom_ylim[1])
        df_plot = data[mask]

        im = ax.scatter(df_plot.Longitude, df_plot.Latitude,
                        c=df_plot[col_name],
                        cmap=custom_cmap, vmin=0, vmax=v_max,
                        s=15, marker='s', zorder=1, alpha=0.8)

        # Solid Outlines (alpha=1.0)
        gdf_gem.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5, alpha=1.0, zorder=2)

    # Landslide Overlay (Target=1 from landslides.csv)
    if overlay_events:
        df_ls = pd.read_csv(train_file)
        if 'Target' in df_ls.columns:
            events = df_ls[df_ls['Target'] == 1]
            # Clip to map view
            events = events[
                (events.Longitude >= zoom_xlim[0]) & (events.Longitude <= zoom_xlim[1]) &
                (events.Latitude >= zoom_ylim[0]) & (events.Latitude <= zoom_ylim[1])
            ]
            ax.scatter(events.Longitude, events.Latitude,
                       c='black', marker='x', s=35, linewidth=1.0,
                       label='Recorded Landslides', zorder=3)
            ax.legend(loc='upper right', fontsize=9, framealpha=0.9, edgecolor='black')

    # Colorbar
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label(legend_label, rotation=270, labelpad=15)

    # Styling
    ax.set_xlim(zoom_xlim)
    ax.set_ylim(zoom_ylim)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_aspect(geo_factor)

    save_path = os.path.join(map_dir, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved: {filename}")

# EXECUTE MAPS
# 1. Historical
if 'Historical' in data_frames:
    plot_final_layout(data_frames['Historical'], 'Landslide_Probability',
                      "Historical Susceptibility (2015-2024)", "Map_Historical_new.png",
                      legend_label='Landslide Probability', overlay_events=True)

# 2. Scenarios
for name in ['SSP1-2.6', 'SSP3-7.0', 'SSP5-8.5']:
    if name in data_frames:
        code = name.replace('-', '').replace('.', '')
        plot_final_layout(data_frames[name], 'Landslide_Probability',
                          f"Future Projection: {name} (2081-2100)", f"Map_{code}_new.png",
                          legend_label='Landslide Probability')

# 3. Municipality (SSP3)
if 'SSP3-7.0' in data_frames:
    df_grid = data_frames['SSP3-7.0']
    gdf_points = gpd.GeoDataFrame(df_grid, geometry=gpd.points_from_xy(df_grid.Longitude, df_grid.Latitude), crs="EPSG:4326")
    joined = gpd.sjoin(gdf_points, gdf_gem, how="inner", predicate="within")

    col = 'NAME_DE' if 'NAME_DE' in joined.columns else joined.columns[0]
    # Calculate Share > 0.5
    risk_gem = joined.groupby(col)['Landslide_Probability'].apply(lambda x: (x > 0.5).mean()).reset_index()
    gdf_final = gdf_gem.merge(risk_gem.rename(columns={'Landslide_Probability': 'High_Risk_Share'}), on=col)

    plot_final_layout(gdf_final, 'High_Risk_Share',
                      "Municipality High-Risk Zones (SSP3-7.0)",
                      "Municipality_Risk_Share_SSP3_new.png",
                      is_shapefile=True, v_max=0.5,
                      legend_label='Share of High-Risk Area (p>0.5)')

print("\nProcess Complete. All outputs in:", map_dir)

In [ ]:
# block18
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from google.colab import drive

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')

scenarios = {
    'Historical': 'Risk_Map_Data_2015_2024.csv',
    'SSP1-2.6': 'Result_Grid_SSP126.csv',
    'SSP3-7.0': 'Result_Grid_SSP370.csv',
    'SSP5-8.5': 'Result_Grid_SSP585.csv'
}
# UPDATE GRID FILES & GENERATE STATS

files_to_update = {
    'Historical': 'Risk_Map_Data_2015_2024.csv',
    'SSP1-2.6': 'Result_Grid_SSP126.csv',
    'SSP3-7.0': 'Result_Grid_SSP370.csv',
    'SSP5-8.5': 'Result_Grid_SSP585.csv'
}

data_frames = {}
stats_list = []

for name, filename in files_to_update.items():
    path = os.path.join(map_dir, filename)
    if os.path.exists(path):
        print(f"Updating {name} ({filename})...")
        df_grid = pd.read_csv(path)

        # Check if features exist
        if all(f in df_grid.columns for f in features):
            probs = rf.predict_proba(df_grid[features])[:, 1]
            df_grid['Landslide_Probability'] = probs

            # Add 'Scenario' column for Violin Plot
            df_grid['Scenario'] = name

            # Save back to drive
            df_grid.to_csv(path, index=False)
            data_frames[name] = df_grid

            # Calculate Statistics for the Table
            mean_risk = probs.mean()
            high_risk_share = (probs > 0.5).mean()

            stats_list.append({
                'Scenario': name,
                'Mean_Probability': mean_risk,
                'High_Risk_Share_Percent': high_risk_share * 100
            })
        else:
            print(f"  -> ERROR: Missing features in {filename}.")
    else:
        print(f"  -> WARNING: File {filename} not found. Cannot update or plot.")

# SAVE STATISTICS TABLE
if stats_list:
    stats_df = pd.DataFrame(stats_list)
    stats_out = os.path.join(map_dir, 'Scenario_Statistics_Table.csv')
    stats_df.to_csv(stats_out, index=False)
    print(f"\n-> Statistics Table saved to: {stats_out}")
    print(stats_df.to_string(index=False))

# PLOT: Violin Plot
if data_frames:
    print("\nGenerating Violin Plot...")
    # Combine data for plotting
    df_all = pd.concat([d[['Landslide_Probability', 'Scenario']] for d in data_frames.values()])

    plt.figure(figsize=(12, 8))
    # Colors: Gray (Hist), Green (SSP1), Orange (SSP3), Red (SSP5)
    palette = ['#A9A9A9', '#009900', '#FF8C00', '#FF0000']

    sns.violinplot(data=df_all, x='Scenario', y='Landslide_Probability',
                   palette=palette, inner='quartile', linewidth=1.5)

    plt.title('Distribution of Landslide Risk (Violin Plot)', fontsize=15)
    plt.ylabel('Landslide Probability (0-1)', fontsize=12)
    plt.ylim(0, 1)
    plt.grid(axis='y', alpha=0.3)
    plt.savefig(os.path.join(map_dir, 'risk_distribution_violin.png'), dpi=300)
    plt.close()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import os
import geopandas as gpd
from google.colab import drive

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Landslides'
map_dir = os.path.join(base_dir, 'Output_Maps')

# Results in numbers

# 1. Confusion Matrix
try:
    print("\n MODELLPERFORMANZ")
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"Wahr-Negativ (Sicher als Sicher erkannt): {cm[0][0]}")
    print(f"Falsch-Positiv (Fehlalarm): {cm[0][1]}")
    print(f"Falsch-Negativ (Erdrutsch übersehen): {cm[1][0]}")
    print(f"Wahr-Positiv (Erdrutsch korrekt erkannt): {cm[1][1]}")
    print("\nKlassifikationsbericht:")
    print(classification_report(y_test, y_test_pred))
except NameError:
    print("Fehler: y_test nicht gefunden.")

# 2. Feature Importance Ranking
try:
    print("\n FEATURE IMPORTANCE RANKING")
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]
    for i in range(len(features)):
        print(f"{i+1}. {features[indices[i]]}: {importances[indices[i]]:.4f} ({importances[indices[i]]*100:.1f} generalised%) ")
except NameError:
    print("Fehler: Das Modell 'rf' wurde nicht gefunden.")

# 3. Räumliche Hotspots (Gemeinde-Auswertung Historisch vs. SSP5-8.5)
try:
    print("\n RÄUMLICHE HOTSPOTS (GEMEINDEN)")

    # Shapefile laden
    gdf_gem = gpd.read_file(shapefile_path).to_crs("EPSG:4326")
    col = 'NAME_DE' if 'NAME_DE' in gdf_gem.columns else gdf_gem.columns[0]

    # Historisch berechnen
    df_hist = pd.read_csv(os.path.join(map_dir, 'Risk_Map_Data_2015_2024.csv'))
    gdf_hist = gpd.GeoDataFrame(df_hist, geometry=gpd.points_from_xy(df_hist.Longitude, df_hist.Latitude), crs="EPSG:4326")
    joined_hist = gpd.sjoin(gdf_hist, gdf_gem, how="inner", predicate="within")
    risk_hist = joined_hist.groupby(col)['Landslide_Probability'].apply(lambda x: (x > 0.5).mean() * 100).reset_index()
    risk_hist.columns = [col, 'Gefahrenzone_Historisch_Prozent']

    # SSP3-7.0
    df_ssp3 = pd.read_csv(os.path.join(map_dir, 'Result_Grid_SSP370.csv'))
    gdf_ssp3 = gpd.GeoDataFrame(df_ssp3, geometry=gpd.points_from_xy(df_ssp3.Longitude, df_ssp3.Latitude), crs="EPSG:4326")
    joined_ssp3 = gpd.sjoin(gdf_ssp3, gdf_gem, how="inner", predicate="within")
    risk_ssp3 = joined_ssp3.groupby(col)['Landslide_Probability'].apply(lambda x: (x > 0.5).mean() * 100).reset_index()
    risk_ssp3.columns = [col, 'Gefahrenzone_SSP3_Prozent']

    # Vergleichen
    comparison = pd.merge(risk_hist, risk_ssp3, on=col)
    comparison['Zunahme_Prozentpunkte'] = comparison['Gefahrenzone_SSP3_Prozent'] - comparison['Gefahrenzone_Historisch_Prozent']

    print("\nTop 3 Gemeinden in der Historischen Baseline (Höchster Anteil an Zonen > 0.5):")
    print(comparison.sort_values(by='Gefahrenzone_Historisch_Prozent', ascending=False)[[col, 'Gefahrenzone_Historisch_Prozent']].head(3).to_string(index=False))

    print("\nTop 3 Gemeinden mit dem stärkten Anstieg im Szenario SSP3-7.0:")
    print(comparison.sort_values(by='Zunahme_Prozentpunkte', ascending=False)[[col, 'Gefahrenzone_Historisch_Prozent', 'Gefahrenzone_SSP3_Prozent', 'Zunahme_Prozentpunkte']].head(3).to_string(index=False))

except Exception as e:
    print(f"\nInfo: Gemeinde-Auswertung übersprungen (Fehler oder Dateien fehlen: {e})")